> 请在环境变量中设置`DB_URI`指向数据库

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from alphamind.api import *
from alphamind.strategy.strategy import RunningSetting
from alphamind.strategy.strategy import Strategy
from PyFin.api import *
from PyFin.Math.Accumulators.StatefulAccumulators import MovingAverage
from PyFin.Math.Accumulators.StatefulAccumulators import MovingSharp
from PyFin.Math.Accumulators.StatefulAccumulators import MovingMaxDrawdown

plt.style.use('ggplot')

In [ ]:
"""
Back test parameter settings
"""

benchmark_code = 905
universe = Universe('zz800') + Universe('cyb')

start_date = '2017-01-01'
end_date = '2018-04-27'
freq = '10b'
neutralized_risk = None

alpha_factors = {
    'f01': CSQuantiles(LAST('EPS'), groups='sw1'),
    'f02': CSQuantiles(LAST('ROE'), groups='sw1'),
}

weights = dict(
               f01=1.,
               f02=1.
               )

alpha_model = ConstLinearModel(features=alpha_factors, weights=weights)

data_meta = DataMeta(freq=freq,
                     universe=universe,
                     batch=1,
                     neutralized_risk=None,
                     pre_process=None,
                     post_process=None,
                     data_source=os.environ['DB_URI'])

strategy = Strategy(alpha_model,
                    data_meta,
                    universe=universe,
                    start_date=start_date,
                    end_date=end_date,
                    freq=freq,
                    benchmark=benchmark_code)

strategy.prepare_backtest_data()

def create_scenario(weights_bandwidth=0.02, target_vol=0.01, method='risk_neutral'):
    industry_names = industry_list('sw', 1)
    constraint_risk = ['SIZE', 'SIZENL', 'BETA']
    total_risk_names = constraint_risk + ['benchmark', 'total']
    all_styles = risk_styles + industry_styles + macro_styles

    b_type = []
    l_val = []
    u_val = []

    previous_pos = pd.DataFrame()
    rets = []
    turn_overs = []
    leverags = []

    for name in total_risk_names:
        if name == 'benchmark':
            b_type.append(BoundaryType.RELATIVE)
            l_val.append(0.8)
            u_val.append(1.0)
        else:
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(0.0)
            u_val.append(0.0)

    bounds = create_box_bounds(total_risk_names, b_type, l_val, u_val)
    running_setting = RunningSetting(weights_bandwidth=weights_bandwidth,
                                     rebalance_method=method,
                                     bounds=bounds,
                                     target_vol=target_vol,
                                     turn_over_target=0.4)

    ret_df, positions = strategy.run(running_setting)
    return ret_df

In [ ]:
def create_report(ret_df, windows):
    sharp_calc = MovingSharp(windows, x='ret', y='riskFree')
    drawdown_calc = MovingMaxDrawdown(windows, x='ret')
    max_drawdown_calc = MovingMaxDrawdown(len(ret_df), x='ret')
    
    ret_df['ret_after_tc'] = ret_df.excess_return - 0.002 * ret_df.turn_over
    res_df = pd.DataFrame(columns=['daily_return', 'cum_ret', 'sharp', 'drawdown', 'max_drawn', 'leverage'])
    total_returns = 0.

    for i, ret in enumerate(ret_df['ret_after_tc']):
        date = ret_df.index[i]
        total_returns += ret
        sharp_calc.push({'ret': ret, 'riskFree': 0.})
        drawdown_calc.push({'ret': ret})
        max_drawdown_calc.push({'ret': ret})

        res_df.loc[date, 'daily_return'] = ret
        res_df.loc[date, 'cum_ret'] = total_returns
        res_df.loc[date, 'drawdown'] = drawdown_calc.result()
        res_df.loc[date, 'max_drawn'] = max_drawdown_calc.result()
        res_df.loc[date, 'leverage'] = ret_df.loc[date, 'leverage']

        if i < 5:
            res_df.loc[date, 'sharp'] = 0.
        else:
            res_df.loc[date, 'sharp'] = sharp_calc.result() * np.sqrt(windows)
    return res_df

In [ ]:
%%time
weight_gaps = [0.005, 0.010, 0.015, 0.020]

with pd.ExcelWriter(f'zz800_cyb_{benchmark_code}_gap.xlsx', engine='xlsxwriter') as writer:
    for i, weight_gap in enumerate(weight_gaps):
        ret_df = create_scenario(weight_gap, target_vol=0.01, method='risk_neutral')
        res_df = create_report(ret_df, 25)
        res_df.to_excel(writer, sheet_name=f'{i}')
        alpha_logger.info(f"weight_gap: {weight_gap} finished")

In [ ]:
%%time
target_vols = [0.015, 0.030, 0.045, 0.060]

with pd.ExcelWriter(f'zz800_cyb_{benchmark_code}_tv.xlsx', engine='xlsxwriter') as writer:
    for i, target_vol in enumerate(target_vols):
        ret_df = create_scenario(0.01, target_vol=target_vol, method='tv')
        res_df = create_report(ret_df, 25)
        res_df.to_excel(writer, sheet_name=f'{i}')
        alpha_logger.info(f"target_vol: {target_vol:.4f} finished")